## Heavily adapted from 
[here](https://github.com/pytorch/examples/blob/master/vae/main.py)

In [15]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
batch_size=128
epochs = 1000
log_interval = 100

In [18]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.ToTensor()
                  ),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [19]:
im_dm=28*28
z_dim=20

In [28]:
im_dm

784

In [35]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        ## Encode
        self.fce1 = nn.Linear(im_dm, im_dm)
        self.fce2 = nn.Linear(im_dm, im_dm)
        self.fce3 = nn.Linear(im_dm, im_dm)
        self.fce4 = nn.Linear(im_dm, 400)
        # Sigma2, Mu
        self.fce51 = nn.Linear(400, z_dim)
        self.fce52 = nn.Linear(400, z_dim)
        
        ## Decode
        self.fcd1 = nn.Linear(z_dim, 400)
        self.fcd2 = nn.Linear(400, im_dm)
        self.fcd3 = nn.Linear(im_dm, im_dm)
        self.fcd4 = nn.Linear(im_dm, im_dm)
        self.fcd5 = nn.Linear(im_dm, im_dm)
        #self.fc4 = nn.Linear(800, 80*80)

    def encode(self, x):
        #eh1 = torch.sigmoid(self.fce1(x))
        eh1 = torch.sigmoid(self.fce4(x))
        #eh2 = torch.sigmoid(self.fce2(eh1))
        #eh3 = torch.sigmoid(self.fce3(eh2))
        #eh4 = torch.sigmoid(self.fce4(eh3))
        #return self.fce51(eh4), self.fce52(eh4)
        return self.fce51(eh1), self.fce52(eh1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        #dh1 = torch.sigmoid(self.fcd1(z))
        dh1 = torch.sigmoid(self.fcd1(z))
        #dh2 = torch.sigmoid(self.fcd2(dh1))
        #dh3 = torch.sigmoid(self.fcd3(dh2))
        #dh4 = torch.sigmoid(self.fcd4(dh3))
        #return torch.relu(self.fcd5(dh4))
        #return torch.sigmoid(self.fcd5(dh4))
        return torch.sigmoid(self.fcd2(dh1))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, im_dm))
        #mu, logvar = self.encode(x.view(-1, 80*80))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [36]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [37]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, beta=1):
    #BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    L1 = F.l1_loss(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return L1 + beta * KLD

In [38]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar, beta=0.000001)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [39]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar, beta=0.001).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [41]:
## Main
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    if epochs % 10 == 0:
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.000425
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.000462
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.000415
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.000448
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.000429
====> Epoch: 1 Average loss: 0.0004
====> Test set loss: 0.0442
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.000397
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.000434
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.000420
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.000401
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.000430
====> Epoch: 2 Average loss: 0.0004
====> Test set loss: 0.0449
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.000406
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.000400
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.000412
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.000380
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.000382
====> Epoch: 3 Average loss: 0.0004
====> Test set loss: 0.0459
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.000413
Train Epoch: 4 [12800/60000 

KeyboardInterrupt: 

In [65]:
28*28

784